In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
shared_index = set()
shared_columns = set()
collective_frames = {}
for root, dirs, files in os.walk("../third-party/Brazilian Weather/"):
    individual_frames = []
    for filename in files:
        if filename[-3:] == "WTH":
            dataframe = pd.read_csv("%s/%s" % (root, filename), skiprows=4, sep='\s+', index_col=0)
            individual_frames.append(dataframe)  # all frames are in the same folder
    if len(individual_frames) > 0:
        dataframe = pd.concat(individual_frames, verify_integrity=True).sort_index()
        dataframe = dataframe.dropna(axis='columns')
        shared_index = set(dataframe.index) if len(shared_index) == 0 else shared_index & set(dataframe.index)
        shared_columns = set(dataframe.columns) if len(shared_columns) == 0 else shared_columns & set(dataframe.columns)
        collective_frames[root[-4:]] = dataframe

In [3]:
shared_index = sorted(shared_index)
shared_columns = sorted(shared_columns)

In [4]:
dataframes = []
for key in collective_frames.keys():
    dataframe = collective_frames[key].loc[shared_index]
    dataframe = dataframe[shared_columns]
    dataframes.append(dataframe.values)
matrix = np.stack(dataframes)

In [5]:
shared_index = [str(idx) for idx in shared_index]
shared_columns = [str(idx) for idx in shared_columns]

In [6]:
np.save("../prepared-data/Brazilian Weather (%dx%dx%d)" % matrix.shape, matrix)

In [7]:
with open("../prepared-data/Brazilian Weather (%dx%dx%d).idx" % matrix.shape,'w') as f:
    f.write('\n'.join(shared_index))

In [8]:
with open("../prepared-data/Brazilian Weather (%dx%dx%d).col" % matrix.shape,'w') as f:
    f.write('\n'.join(shared_columns))